In [1]:
from learning_pipelines import get_learning_results, get_hyper_param_results
import sklearn.svm as svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
import pandas as pd

from classification.keras_neural_nets import *

import os

/home/franciscosargo/.virtualenvs/pre_epi_seizures/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


SyntaxError: EOL while scanning string literal (iopes.py, line 91)

In [ ]:
# arguments
label_struct = {
                        'inter_ictal':{
                            'label': 'Inter-Ictal Data Points',
                            'color': 'blue',
                            'intervals_samples': [(1000 * 0 * 60, 1000 * 0 * 60 )]
                                     },
                        'pre_ictal':{
                            'label': 'Pre-Ictal data points',
                            'color': 'yellow',
                            'intervals_samples': [(1000 * 20 * 60, 1000 * 49 * 60 )]
                                     },
                        'ictal':{
                            'label': 'Ictal data points',
                            'color': 'red',
                            'intervals_samples': [(1000 * 50 * 60, 1000 * 55 * 60 )]
                                 },
                        }
baseline_label_struct = {
                         'baseline':{
                            'label': 'Baseline Data Points',
                            'color': 'green',
                            'intervals_samples': [(0, 1000 * 60 * 60 )]
                                    },
                        }



param_grid = [
              {'ANN__epochs': [800],
               'ANN__batch_size': [10**4],
               'ANN__hidden_layers_nr': [1],
               'ANN__hidden_nodes_per_layer': [i for i in xrange(3, 13)],
               'ANN__hidden_nodes_activation': ['relu']
              }
]

pipe = Pipeline( [('ANN', KerasClassifier(build_fn=dense_network,
                                        input_dim = 5,
                                          output_dim = 3,
                                          verbose=0))])

#param_grid = [
#               {'GaussNB__priors': [None]},
#                  ]
#pipe = Pipeline([('GaussNB', GaussianNB())])

param_grid = [
               {'SVC__C': [2**i for i in xrange(-5, 11)],
                'SVC__gamma':[2**i for i in xrange(-15, 1)]},
                 ]

pipe = Pipeline([('SVC', svm.SVC())])


feature_slot = 'pca_beat_amp_computation'
hyper_param=0


scaler = StandardScaler()

plot_eda=False
learn_flag=True
compute_all_new=True

In [ ]:
# Perform Hyper analysis

patient_lists = [[3], [4]]




table = [get_learning_results(label_struct, baseline_label_struct,
                                              pipe, scaler, param_grid,
                                              patient_list,
                                              feature_slot,
                                              hyper_param,
                                              plot_eda,
                                              learn_flag,
                                              compute_all_new)
                   for patient_list in patient_lists]

hp_opt_results = [get_hyper_param_results(label_struct, baseline_label_struct,
                                              pipe, scaler, param_grid,
                                              patient_list,
                                              feature_slot,
                                              hyper_param,
                                              plot_eda,
                                              learn_flag,
                                              compute_all_new)
                   for patient_list in patient_lists]
    

In [ ]:
table = pd.concat(table)
table

In [ ]:
table

In [ ]:
# Overal Patient Classification
per_patient = table[['precision',
                     'recall',
                     'f1-score',
                     'patient_nr',
                     'Best Parameters',
                     'Labels']].groupby(['patient_nr', 'Labels']).describe()

per_patient.loc[:, (['precision', 'recall', 'f1-score'], ['mean', 'std'])]

#print per_patient.columns.get_level_values(1)=='std'

#per_patient = per_patient.iloc[:,
                               #per_patient.columns.get_level_values(1)=='std']
#per_patient

In [ ]:
# Overal Patient Classification
per_patient = table[['precision',
                     'recall',
                     'f1-score',
                     'patient_nr',
                     'Best Parameters',
                     'Labels']].groupby(['Best Parameters','patient_nr', 'Labels']).describe()

per_patient.loc[:, (['precision', 'recall', 'f1-score'], ['mean', 'std'])]

In [ ]:
# 

per_patient = table[['precision',
                     'recall',
                     'f1-score',
                     'patient_nr',
                     'Labels',
                     'types_of_seizure',
                     'location']].groupby(['patient_nr',
                                           'Labels',
                                           'types_of_seizure',
                                           'location']).describe()
per_patient

In [ ]:
per_patient.loc[:, (['precision', 'recall', 'f1-score'], ['mean', 'std'])]



In [ ]:
from classification.hp_opt import heat_new




map(heat_new.get_hp_opt_results_args, hp_opt_results)


In [ ]:
hp_opt_results